# Classification Experiment on Stanforf EEG Imagined speech dataset

Run this experiment to obtain a similar result as the one publisher in the paper

```latex
@article{gallo2024eeg,
  title={Thinking is Like a Sequence of Words},
  author={Gallo, Ignzio and Coarsh, Silvia},
  journal={IJCNN},
  volume={??},
  pages={??--??},
  year={2024},
  publisher={IEEE}
}
```

## Import libraries

In [1]:
import os
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, ConcatDataset
import yaml
import mne
import random
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
import scipy.io
from sklearn.model_selection import KFold

In [2]:
def create_save_dir(args):
    if "subject_num" in args:
        if type(args['subject_num']) is not list:
            args['subject_num'] = [args['subject_num']]

        subjs_str = ','.join(str(x) for x in args['subject_num'])
        args['save_dir'] = os.path.join(args['save_dir'], subjs_str, datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    else:
        args['save_dir'] = os.path.join(args['save_dir'], datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
    if not os.path.isdir(args['save_dir']):
        os.makedirs(args['save_dir'])
    print("Saving results in:", args['save_dir'])

In [3]:
class Metrics:
    def __init__(self, column_names):
        column_names.insert(0, "time_stamp")
        self.df = pd.DataFrame(columns=column_names)

    def add_row(self, row_list):
        row_list.insert(0, str(dt.datetime.now()))
        # print(row_list)
        self.df.loc[len(self.df)] = row_list

    def save_to_csv(self, filepath):
        self.df.to_csv(filepath, index=False)

## Load raw dataset

dataset: https://purl.stanford.edu/bq914sc3730

paper using this dataset: https://www.sciencedirect.com/science/article/pii/S0031320322002382?casa_token=uoQfTQnrCZoAAAAA:P6d4nfYk9FUyy6gTpGNJM6eAINAjuNaxnzwf-5RNNLu9ubl7B6WY3YDjMEaz38SKfIgxalScAfs#sec0012


In [4]:
def read_stanford_trts(args, sub):
    SCALE = 0.1
    #for sub in range(1,11):
    mat = scipy.io.loadmat(os.path.join(args['data_dir'],f'S{sub}.mat'))  
    X = mat['X_3D'] # (electrodes, time, trials)
    X = np.transpose(X, (2, 1, 0)) # transform into (trials, time, electrodes)
    X *= SCALE
    Y = mat['categoryLabels'][0] # [1 2 3 4 5 6] different categories   
    X_new = X #np.concatenate([X_new, X]) if X_new.size else X
    Y_new = Y #np.concatenate([Y_new, Y]) if Y_new.size else Y

    mean, std = X_new.mean(), X_new.std()
    print("Input data: mean =", mean, ", std =", std) # used for input data Normalization
    print("Stanford Dataset shape:", X_new.shape, ", min:", np.min(X_new), ", max:", np.max(X_new))
    Y_new = Y_new - 1 # [0 1 2 3 4 5] category indexes 
    print("category indexes:", np.unique(Y_new))
    
    # Random split. 
    X_train, X_test, Y_train, Y_test = train_test_split(np.float32(X_new), Y_new.astype(int), test_size=0.1, random_state=1)   

    # Convert NumPy arrays to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    Y_train = torch.tensor(Y_train, dtype=torch.long)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    Y_test = torch.tensor(Y_test, dtype=torch.long)

    # Convert data to DataLoader
    train_dataset = TensorDataset(X_train, Y_train)
    train_loader = DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True)
    test_dataset = TensorDataset(X_test, Y_test)
    test_loader = DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False)
    return train_loader, test_loader

## The model

Deep neural network based on a basic Transformer.

In [5]:
class NetTraST(nn.Module):
    def __init__(self, args): 
        super(NetTraST, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(args['vocab_size'])
        p = args['kernel_size'] // 2
        self.conv1 = nn.Conv1d(in_channels=args['vocab_size'], out_channels=args['kernel_num'], kernel_size=args['kernel_size'], stride=1, padding=p)
        
        self.conv2 = nn.Conv1d(in_channels=args['embed_dim'], out_channels=args['kernel_num'], kernel_size=args['kernel_size'], stride=1, padding=p)
        self.upsamp = nn.Upsample((args['embed_dim']))
        
        self.rrelu = nn.RReLU(0.1, 0.3)
        nl=3 #args['num_layers']//2
        self.spatial_tra = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=args['embed_dim'],
                nhead=args['nhead'],
                dim_feedforward=args['dim_feedforward'],
            ),
            num_layers=nl,
        )
        self.temporal_tra = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=args['vocab_size'],
                nhead=args['nhead'],
                dim_feedforward=args['dim_feedforward'],
            ),
            num_layers=nl,
        )
        """
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=args['kernel_num'],
                nhead=args['nhead'],
                dim_feedforward=args['dim_feedforward'],
            ),
            num_layers=args['num_layers'],
        )
        """
        self.batch_norm3 = nn.BatchNorm1d(args['kernel_num'])
        self.fl = nn.Flatten()
        self.fc1 = nn.Linear(args['kernel_num']*args['embed_dim'], args['kernel_num'])
        self.dropout = nn.Dropout(args['dropout'])
        self.fc2 = nn.Linear(args['kernel_num'], args['class_num'])

    def forward(self, x): 
        x = self.batch_norm1(x)
        
        x1 = self.conv1(x) 
        x1 = self.spatial_tra(x1)
        
        x2 = x.permute(0, 2, 1)
        x2 = self.conv2(x2) 
        x2 = self.temporal_tra(x2)
        x2 = self.upsamp(x2)

        x = x1+x2 
        #x = torch.cat((x1, x2), 1)
        
        # Reshape the input for the Transformer layer
        #x = x.permute(2, 0, 1)  # Change the shape to (sequence_length, batch_size, input_size)
        #x = self.transformer(x)
        # Reshape the output back to the original shape
        #x = x.permute(1, 2, 0)  # Change the shape to (batch_size, input_size, sequence_length)
        x = self.batch_norm3(x)
        x = self.fl(x)
        x = self.rrelu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

## Training and Evaluation functions

In [6]:
def evaluation_raw(args, model, test_loader, criterion):
    # Evaluation
    model.eval()
    with torch.no_grad():
        tot_loss = 0
        test_corrects = torch.tensor(0, device=args['device'])
        for inputs, labels in test_loader:
            inputs = inputs.to(args['device'])
            labels = labels.to(args['device'])
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            corrects = (torch.max(outputs, 1)[1].view(labels.size()).data == labels.data).sum()
            test_corrects += corrects
            tot_loss += loss

        ts_acc = 100.0 * test_corrects/len(test_loader.dataset)
    return ts_acc.cpu().item(), tot_loss


def train_raw(args, model, train_loader, optimizer, criterion, test_loader, metrics, fold, scheduler=None): 
    best_acc = 0
    patience_counter = 0
    steps = 0
    loop_obj = tqdm(range(args['epochs']))
    loop_obj.set_postfix_str(f"Best val. acc.: {best_acc:.4f}")  # Adds text after progressbar
    for epoch in loop_obj:
        loop_obj.set_description(f"Subj.: {fold}, Training epoch: {epoch+1}")  # Adds text before progessbar
        train_corrects = torch.tensor(0, device=args['device'])
        tot_loss = 0
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            inputs = inputs.to(args['device'])

            labels = labels.to(args['device'])
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            corrects = (torch.max(outputs, 1)[1].view(labels.size()).data == labels.data).sum()
            train_corrects += corrects
            tot_loss += loss
            loss.backward()
            optimizer.step()

        if scheduler: scheduler.step()

        tr_acc = 100.0 * train_corrects/len(train_loader.dataset)
        # Validation
        dev_acc, test_loss = evaluation_raw(args, model, test_loader, criterion)

        if dev_acc > best_acc:
            best_acc = dev_acc
            patience_counter = 0
            loop_obj.set_postfix_str(f"Best val. acc.: {best_acc:.4f}")
        else:
            patience_counter += 1

        lr=optimizer.param_groups[0]["lr"]
        metrics.add_row([epoch+1, lr, tot_loss.cpu().item(), tr_acc.cpu().item(), test_loss.cpu().item(), dev_acc, best_acc])
        metrics.save_to_csv(os.path.join(args['save_dir'], "metrics_classifciation.csv"))

        if patience_counter >= args['early_stopping_patience']:
            print("Early stopping...")
            break
    return best_acc

## Run an experiment

- change the parameter '*data_dir*' in the *args* dictionary 
- remenber to change also the '*save_dir*' parameter

We evaluated our models for only one classification task, namely, 
- 6-class category-level classification, 

In [7]:
def get_default_args():
    args = {
        'class_num': 6,
        'dropout': 0.1 ,
        'nhead': 4 , 
        'dim_feedforward': 128 ,
        'num_layers': 14, 
        'embed_dim': 124,
        'vocab_size': 32,
        'kernel_num': 56, 
        'kernel_size': 3, 
        'batch_size': 64, # use low batch size
        'epochs': 1000 ,
        'early_stopping_patience': 300,
        'lr': 0.9 ,
        'log_interval': 1,
        'device': 'cuda:1', # cuda, cpu
        'data_dir': '/home/jovyan/nfs/igallo/datasets/EEG/stanford/', # S1.mat, ..., S10.mat
        'save_dir': 'experiments/transformer/stanford/', # 
        'save_best': True,
        'verbose': True,
        'test_interval': 100,
        'save_interval': 500,
        'sampling_rate': 62.5,
        'k_folds': 10,
    }
    # os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    # os.environ["CUDA_VISIBLE_DEVICES"] = args['gpus']
    return args

- 'num_layers': 14, for the final transformer
- 'num_layers': 3, for the first two transformers
- 'kernel_num': 56, 
- 'epochs': 1000 ,
- 'early_stopping_patience': 300,

### Ablation 5
- NO Transformer T3

In [8]:
def single_run():
    args = get_default_args()
    create_save_dir(args)
    with open(os.path.join(args['save_dir'], "config.yaml"), "w") as f:
        yaml.dump(
            args, stream=f, default_flow_style=False, sort_keys=False
        )

    # For all the results
    metrics = Metrics(["epoch", "lr", "train_loss", "train_acc", "test_loss", "test_acc", "best_test_acc"])
    results = {}
    for sub in range(1,11):
        model = NetTraST(args)
        model = model.to(args['device'])

        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters()) #, lr=args['lr'])

        train_loader, test_loader = read_stanford_trts(args, sub) 
        print("Training size:", len(train_loader.dataset))
        print("Test size:", len(test_loader.dataset))

        best_acc = train_raw(args, model, train_loader, optimizer, criterion, test_loader, metrics, sub) 
        #torch.save(model, os.path.join(args['save_dir'], f"{model.__class__.__name__}_model_last.pt"))
        results[sub] = best_acc
        accs = np.array(list(results.values()))
        print(f'acc: mean={np.mean(accs):.4f}%, std={np.std(accs):.4f}%')

    # Print subject results
    str = f"RESULTS FOR 10 SUBJECTS\n"
    str += '--------------------------------\n'
    for key, value in results.items():
        str += f'Subject {key}: {value:.4f} %\n'
    accs = np.array(list(results.values()))
    str += f'mean: {np.mean(accs):.4f}%, std: {np.std(accs):.4f}%\n'
    print(str)
    with open(os.path.join(args['save_dir'], "mean_results.txt"), "w") as f:
        f.write(str)

In [ ]:
single_run()

Saving results in: experiments/transformer/stanford/2023-12-11_09-05-26
Input data: mean = 1.7321293941255597e-20 , std = 0.015957179516901867
Stanford Dataset shape: (5188, 32, 124) , min: -1.2147637743651565 , max: 3.0813462468876276
category indexes: [0 1 2 3 4 5]
Training size: 4669
Test size: 519


Subj.: 1, Training epoch: 309:  31%|███       | 308/1000 [10:13<22:59,  1.99s/it, Best val. acc.: 47.2062]

Early stopping...
acc: mean=47.2062%, std=0.0000%


Input data: mean = 6.64409829780833e-23 , std = 0.009863266299745401
Stanford Dataset shape: (5185, 32, 124) , min: -1.0336102391247837 , max: 1.4079203544836336
category indexes: [0 1 2 3 4 5]
Training size: 4666
Test size: 519


Subj.: 2, Training epoch: 302:  30%|███       | 301/1000 [14:29<33:39,  2.89s/it, Best val. acc.: 43.7380]

...

Input data: mean = 2.7507241480672456e-21 , std = 0.01296664363084747
Stanford Dataset shape: (5185, 32, 124) , min: -1.0386944604699646 , max: 1.3983891328368285
category indexes: [0 1 2 3 4 5]
Training size: 4666
Test size: 519


Subj.: 5, Training epoch: 489:  49%|████▉     | 488/1000 [12:18<12:54,  1.51s/it, Best val. acc.: 58.7669]

Early stopping...
acc: mean=49.7495%, std=5.3053%


Input data: mean = 1.2162762139243417e-21 , std = 0.010993911972572074
Stanford Dataset shape: (5186, 32, 124) , min: -2.207850458413234 , max: 2.1549545400402357
category indexes: [0 1 2 3 4 5]
Training size: 4667
Test size: 519


Subj.: 6, Training epoch: 141:  14%|█▍        | 140/1000 [03:26<20:37,  1.44s/it, Best val. acc.: 62.8131]